How to add two floating point numbers:

$$\newcommand{\fs}{\mathrm{fs}}$$
$$a_1 = 2^{e_1} \left( 1 + \frac{f_1}{2^{\fs_1}} \right)$$
$$a_2 = 2^{e_2} \left( 1 + \frac{f_2}{2^{\fs_2}} \right)$$

WLOG assume $e_1 \ge e_2$

$$a_1 + a_2 = 2^{e_1} \left[ 1 + \frac{f_1}{2^{\fs_1}} + 2^{e_2-e_1} \left( 1 + \frac{f_2}{2^{\fs_2}} \right) \right]$$

Take a common fs, called  $m := \max \{ \fs_1, \fs_2 \}$, i.e. replace $f_1 / 2^{\fs_1}$ by $f_1.2^{d_1} / 2^m$, where $d_1 := m - \fs_1$. Call the new $f_1$ and $f_2$ by the same names ($f_1$ and $f_2$). [Strictly, $f_1' := f_1. 2^d_1$.] Then

$$a_1 + a_2 = 2^{e_1} \left[ 1 + \frac{f_1}{2^{m}} + 2^{e_2-e_1} \left( 1 + \frac{f_2}{2^{m}} \right) \right]$$

$$a_1 + a_2 = 2^{e_1} \left[ 1 + \frac{f_1}{2^{m}} + \frac{1}{2^{e_1-e_2}} + \frac{f_2}{2^{m + e_1 - e_2}} \right]$$

$$a_1 + a_2 = 2^{e_1} \left[ 1 + \frac{ f_1 . 2^{e_1-e_2} + 2^m + f_2 }{ 2^{m+e_1-e_2} }   \right]$$

If, after cancelling common factors of two,  the denominator is $\le 2^\mathrm{fsizesize}$ then it is exactly representable.

In [2]:
include("SimpleUnums.jl")

+ (generic function with 172 methods)

In [4]:
import Base: +

In [125]:
function +(a1::Unum, a2::Unum)
    
    if a2.e > a1.e
        a1, a2 = a2, a1
    end
    
    e1, f1, fs1 = a1.e, a1.f, a1.fs
    e2, f2, fs2 = a2.e, a2.f, a2.fs
    
    e1 -= (2^a1.esm1 - 1)  # bias
    e2 -= (2^a2.esm1 - 1)
    
    m = max(fs1, fs2)
    
    if m > fs1
        f1 *= (m / fs1)
    end
    
    if m > fs2
        f2 *= (m / fs2)
    end
    
    @show f1, fs1, f2, fs2, e1, e2
    
    new_f = f1*2^(e1-e2) + 2^m + f2
    new_fs = m + e1 - e2
    
    @show new_f, new_fs
    @show 2^e1 * (1 + new_f / 2^new_fs)
    
    if new_f > 2^new_fs
        new_f -= 2^new_fs
        
        new_fs += 1
        e1 += 1
    end
    
    Unum(0, e1 + (2^a1.esm1 - 1), new_f, 0, a1.esm1, new_fs-1)
    
    
end

+ (generic function with 172 methods)

In [126]:
a = Unum(3)

Unum(0,2,1,0,1,0)
s: 0; e: 10; f: 1; u: 0; esm1: 01; fsm1: 00
  3.0↓


In [127]:
b = Unum(6)

Unum(0,5,1,0,2,0)
s: 0; e: 101; f: 1; u: 0; esm1: 10; fsm1: 00
  6.0↓


In [128]:
c = a + b

(f1,fs1,f2,fs2,e1,e2) = (1,1,1,1,2,1)

Unum(0,6,1,0,2,2)
s: 0; e: 110; f: 001; u: 0; esm1: 10; fsm1: 10
  9.0↓



(new_f,new_fs) = (5,2)
2 ^ e1 * (1 + new_f / 2 ^ new_fs) = 9.0


In [120]:
Unum(9)

Unum(0,6,1,0,2,2)
s: 0; e: 110; f: 001; u: 0; esm1: 10; fsm1: 10
  9.0↓


In [130]:
Unum(2.5) + Unum(2.5)

(f1,fs1,f2,fs2,e1,e2) = (1,2,1,2,1,1)

Unum(0,3,2,0,1,2)
s: 0; e: 11; f: 010; u: 0; esm1: 01; fsm1: 10
  5.0↓



(new_f,new_fs) = (6,2)
2 ^ e1 * (1 + new_f / 2 ^ new_fs) = 5.0
